In [317]:
import pandas as pd
import numpy as np
import ast
import json

**Converting JSON into a dataframe and csv**

In [318]:
# Filepath for the json version of the dataset
json_filepath = "yelp_academic_dataset_business.json"

# Reading the JSON file (weird since each line is a separate JSON object)
df = pd.read_json(json_filepath, lines=True)

# Normalizing the nested JSON parts and readding them as multiple columns
df_attributes = pd.json_normalize(df["attributes"])
df_hours = pd.json_normalize(df["hours"])
df = df.drop(["attributes", "hours"], axis=1)
df = pd.concat([df, df_attributes, df_hours], axis=1)

# Saving the dataframe to a csv file
df.to_csv("temp.csv")

**General Features of the Dataset**

Printing out the dimensions, attributes and their types, and first rows of the dataframe. The issues with the columns mentioned above can be seen here.

In [319]:
print(df.shape)
print(df.dtypes)
df.head()

(100000, 58)
business_id                    object
name                           object
address                        object
city                           object
state                          object
postal_code                    object
latitude                      float64
longitude                     float64
stars                         float64
review_count                    int64
is_open                         int64
categories                     object
ByAppointmentOnly              object
BusinessAcceptsCreditCards     object
BikeParking                    object
RestaurantsPriceRange2         object
CoatCheck                      object
RestaurantsTakeOut             object
RestaurantsDelivery            object
Caters                         object
WiFi                           object
BusinessParking                object
WheelchairAccessible           object
HappyHour                      object
OutdoorSeating                 object
HasTV                          object

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,RestaurantsCounterService,AgesAllowed,DietaryRestrictions,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,...,NaN,NaN,NaN,0:0-0:0,8:0-18:30,8:0-18:30,8:0-18:30,8:0-18:30,8:0-14:0,NaN
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,...,NaN,NaN,NaN,8:0-22:0,8:0-22:0,8:0-22:0,8:0-22:0,8:0-23:0,8:0-23:0,8:0-22:0
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,...,NaN,NaN,NaN,7:0-20:0,7:0-20:0,7:0-20:0,7:0-20:0,7:0-21:0,7:0-21:0,7:0-21:0
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,...,NaN,NaN,NaN,NaN,NaN,14:0-22:0,16:0-22:0,12:0-22:0,12:0-22:0,12:0-18:0


**Cleaning the Data**

Let us clean the data by columns. the **is_open** column tells users if the location is closed permanently. We should only analyze the businesses that are still in operation.

In [320]:
df = df[df["is_open"] == 1]
df.drop("is_open", axis=1, inplace=True)
df.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,RestaurantsCounterService,AgesAllowed,DietaryRestrictions,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,...,NaN,NaN,NaN,0:0-0:0,8:0-18:30,8:0-18:30,8:0-18:30,8:0-18:30,8:0-14:0,NaN
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,...,NaN,NaN,NaN,7:0-20:0,7:0-20:0,7:0-20:0,7:0-20:0,7:0-21:0,7:0-21:0,7:0-21:0
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,...,NaN,NaN,NaN,NaN,NaN,14:0-22:0,16:0-22:0,12:0-22:0,12:0-22:0,12:0-18:0
5,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,TN,37015,36.269593,-87.058943,2.0,6,...,NaN,NaN,NaN,0:0-0:0,6:0-22:0,6:0-22:0,6:0-22:0,9:0-0:0,9:0-22:0,8:0-22:0
6,n_0UpQx1hsNbnPUSlodU8w,Famous Footwear,"8522 Eager Road, Dierbergs Brentwood Point",Brentwood,MO,63144,38.627695,-90.340465,2.5,13,...,NaN,NaN,NaN,0:0-0:0,10:0-18:0,10:0-18:0,10:0-18:0,10:0-18:0,10:0-18:0,12:0-18:0


After normalizing the **attributes** and **hours** features, the dataframe gained a significant number of columns (one for each possible attribute and one for each day of the week).

We'll handle the attribute columns first. Let's start with the **ByAppointmentOnly** column.

In [321]:
df["ByAppointmentOnly"].unique()

array([nan, 'False', 'True', 'None'], dtype=object)

We will assume all null values mean it is not by appointment only. This pattern of values is very common in this dataframe, so we will first look for columns with this pattern, and default missing values to False.

In [322]:
pattern = set(df["ByAppointmentOnly"].unique())
matching = []
for col_name in df.columns:
    if set(df[col_name].unique()).issubset(pattern):
        matching.append(col_name)

Now we will replace all null values with False.

In [323]:
mapping = {
    "False" : False,
    "None"  : False,
    "True"  : True
}
for col in matching:
    df[col] = df[col].map(mapping)
    df[col].fillna(False, inplace=True)

C:\Users\kevin\AppData\Local\Temp\ipykernel_23588\2334224800.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(False, inplace=True)
C:\Users\kevin\AppData\Local\Temp\ipykernel_23588\2334224800.py:8: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col].fillna(False, inplace=True)
C:\Users\kevin

We proceed looking for problematic columns.

In [324]:
attribute_cols = df.columns[11:50]
remaining = [x for x in attribute_cols if x not in matching]
remaining

['RestaurantsPriceRange2',
 'WiFi',
 'BusinessParking',
 'Alcohol',
 'RestaurantsAttire',
 'Ambience',
 'NoiseLevel',
 'GoodForMeal',
 'Smoking',
 'Music',
 'BestNights',
 'BYOBCorkage',
 'HairSpecializesIn',
 'AgesAllowed',
 'DietaryRestrictions']

In [325]:
df["RestaurantsPriceRange2"].unique()

array([nan, '1', '2', '3', '4', 'None'], dtype=object)

This column tells us how expensive a restaurant is on a scale from 1-4. We are simply going to fill missing data with the average pricing.

In [326]:
price_mapping = {
    "1" : 1,
    "2" : 2,
    "3" : 3,
    "4" : 4
}

df["RestaurantsPriceRange2"] = df["RestaurantsPriceRange2"].map(price_mapping)
df["RestaurantsPriceRange2"].replace("None", np.nan, inplace=True)
mean = df["RestaurantsPriceRange2"].mean()
df["RestaurantsPriceRange2"].fillna(mean, inplace=True)

remaining.remove("RestaurantsPriceRange2")

C:\Users\kevin\AppData\Local\Temp\ipykernel_23588\1404852848.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["RestaurantsPriceRange2"].replace("None", np.nan, inplace=True)
C:\Users\kevin\AppData\Local\Temp\ipykernel_23588\1404852848.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves 

In [327]:
df["WiFi"].unique()

array([nan, "u'free'", "u'no'", "'free'", "'no'", 'None', "u'paid'",
       "'paid'"], dtype=object)

Let's assume that null values for the **WiFi** column means the venue does not have wifi. Additionally, after research, the *"u"* entries were simply data entered when Yelp used Python 2. We will clean this to only have values of: none, free, paid. There are also *"None"* entries throughout, which we will replace with *"none"*. Additionally, this pattern runs for multiple columns.

In [328]:
def clean_entry(x):
    if isinstance(x, str):
        if (x.startswith("u'") and x.endswith("'")) or (x.startswith('u"') and x.endswith('"')):
            return x[2:-1]
        elif (x.startswith("'") and x.endswith("'")) or (x.startswith('"') and x.endswith('"')):
            return x[1:-1]
        elif x == "None":
            return "none"
    return x

pattern = ["WiFi", "Alcohol", "RestaurantsAttire", "NoiseLevel", "Smoking", "BYOBCorkage", "AgesAllowed"]

for col in pattern:
    df[col] = df[col].apply(clean_entry)
    df[col] = df[col].fillna("none")

rem = [x for x in remaining if x not in pattern]

In [329]:
df["BusinessParking"].unique()

array([nan,
       "{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}",
       "{'garage': None, 'street': None, 'validated': None, 'lot': True, 'valet': False}",
       'None',
       "{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}",
       "{'garage': False, 'street': False, 'validated': False, 'lot': False, 'valet': False}",
       "{'garage': None, 'street': False, 'validated': None, 'lot': True, 'valet': False}",
       "{u'valet': False, u'garage': None, u'street': True, u'lot': False, u'validated': None}",
       "{'garage': True, 'street': False, 'validated': False, 'lot': False, 'valet': False}",
       "{'garage': False, 'street': True, 'validated': False, 'lot': True, 'valet': False}",
       "{'garage': False, 'street': True, 'validated': True, 'lot': True, 'valet': False}",
       "{u'valet': False, u'garage': False, u'street': False, u'lot': True, u'validated': False}",
       "{u'valet': False, u'gara

We will clean this by editing the data so that each entry only includes the type parking it has, and null values will be assumed to have zero parking. The rest of the columns follow this format, so we will clean them together.

In [330]:
def clean_boolean_keys(x):
    x = str(x)
    try:
        parsed = ast.literal_eval(x)
    except Exception:
        return "none"
    
    if not isinstance(parsed, dict):
        return "none"
    
    true_keys = [key for key, value in parsed.items() if value is True]
    return " ".join(true_keys) if true_keys else "none"

for col in rem:
    df[col] = df[col].apply(clean_boolean_keys)

df.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,RestaurantsCounterService,AgesAllowed,DietaryRestrictions,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,...,False,none,none,0:0-0:0,8:0-18:30,8:0-18:30,8:0-18:30,8:0-18:30,8:0-14:0,NaN
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,...,False,none,none,7:0-20:0,7:0-20:0,7:0-20:0,7:0-20:0,7:0-21:0,7:0-21:0,7:0-21:0
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,...,False,none,none,NaN,NaN,14:0-22:0,16:0-22:0,12:0-22:0,12:0-22:0,12:0-18:0
5,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,TN,37015,36.269593,-87.058943,2.0,6,...,False,none,none,0:0-0:0,6:0-22:0,6:0-22:0,6:0-22:0,9:0-0:0,9:0-22:0,8:0-22:0
6,n_0UpQx1hsNbnPUSlodU8w,Famous Footwear,"8522 Eager Road, Dierbergs Brentwood Point",Brentwood,MO,63144,38.627695,-90.340465,2.5,13,...,False,none,none,0:0-0:0,10:0-18:0,10:0-18:0,10:0-18:0,10:0-18:0,10:0-18:0,12:0-18:0


Now let us clean the hours columns. 

In [332]:
def format_time(time):
        hour, minute = time.split(':')
        return f"{int(hour):02d}:{int(minute):02d}"

def convert_hours(x):
    if pd.isna(x) or x == "0:0-0:0":
        return "none"
    
    open, close = x.split('-')
    
    return (format_time(open), format_time(close))

days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

for day in days:
    df[day] = df[day].apply(convert_hours)
     
df.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,RestaurantsCounterService,AgesAllowed,DietaryRestrictions,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,...,False,none,none,none,"(08:00, 18:30)","(08:00, 18:30)","(08:00, 18:30)","(08:00, 18:30)","(08:00, 14:00)",none
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,...,False,none,none,"(07:00, 20:00)","(07:00, 20:00)","(07:00, 20:00)","(07:00, 20:00)","(07:00, 21:00)","(07:00, 21:00)","(07:00, 21:00)"
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,...,False,none,none,none,none,"(14:00, 22:00)","(16:00, 22:00)","(12:00, 22:00)","(12:00, 22:00)","(12:00, 18:00)"
5,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,TN,37015,36.269593,-87.058943,2.0,6,...,False,none,none,none,"(06:00, 22:00)","(06:00, 22:00)","(06:00, 22:00)","(09:00, 00:00)","(09:00, 22:00)","(08:00, 22:00)"
6,n_0UpQx1hsNbnPUSlodU8w,Famous Footwear,"8522 Eager Road, Dierbergs Brentwood Point",Brentwood,MO,63144,38.627695,-90.340465,2.5,13,...,False,none,none,none,"(10:00, 18:00)","(10:00, 18:00)","(10:00, 18:00)","(10:00, 18:00)","(10:00, 18:00)","(12:00, 18:00)"
